In [1]:
from src.utility import get_root
import pandas as pd
import os
from typing import Tuple, List, Optional

In [20]:
def calculate_flops(model: str, input_size: Tuple[int, int], training_strategy: str) -> float:
    path_to_flops = os.path.join(get_root(), "data", "model_flops", "model_flops.xlsx")
    flops_df = pd.read_excel(path_to_flops)

    model_info = flops_df[flops_df["Model"] == model]

    if model_info.empty:
        raise ValueError(f"Model {model} not found in the flops database")
    
    model_type = model_info['Type'].iloc[0]
    original_input_size = model_info['Input Size'].iloc[0].split()[0]

    if model_type == 'Vision':
        width, height = map(int, original_input_size.split('x'))
        scaling = (input_size[0] * input_size[1]) / (width * height)
    else:
        scaling = input_size[0] / int(original_input_size)

    if training_strategy in ["Fine-tuning the whole model", "Full Training"]:
        return model_info['FLOPs'].iloc[0] * scaling * 3
    elif training_strategy == "Last Layer Learning":
        return 2 * model_info['Last Layer FLOPs'].iloc[0] + model_info['FLOPs'].iloc[0] * scaling / 3
    else:
        raise ValueError(f"Unsupported training strategy: {training_strategy}")



a = calculate_flops("bert-base-uncased", (512, ), "Last Layer Learning")
b = calculate_flops("bert-base-uncased", (512, ), "Full Training")

In [21]:
a

14511177728.0

In [22]:
b

130579365888.0

In [23]:
b/a

8.998536737375973